In [1]:
import graphlib
from graphlib import *
import ray
import urllib.request

In [2]:
ray.init()
graph_manager = Graph_collection()

graph_manager.add_graph("CSA_functional_sites")
graph_manager.add_graph("CSA_protein_index")

Waiting for redis server at 127.0.0.1:16765 to respond...
Waiting for redis server at 127.0.0.1:62830 to respond...
Starting local scheduler with 8 CPUs, 0 GPUs

View the web UI at http://localhost:8889/notebooks/ray_ui28490.ipynb?token=6afab869c62d1d8323fae7b123b2d200bbb4b01010fc6f99



In [3]:
count = 0
proteins_set = set()

with open("CSA_2_0_121113.txt", "r") as csa_file:
    header = csa_file.readline().rstrip("\n").split(",")
    for line in csa_file:
        raw_data = line.rstrip("\n").split(",")
        if raw_data[6] != "LIT":
            continue

        count += 1
        proteins_set.add(raw_data[0])
        data = {}
        for i in range(len(raw_data)):
            data[header[i]] = raw_data[i]
        node = Node(data)
        graph_manager.add_node_to_graph("CSA_functional_sites", data["PDB ID"], node)
        
with open("pdb_to_uniprot_csa.txt", "r") as file:
    header = file.readline().rstrip("\n").split("\t")
    
    for line in file:
        raw_data = line.rstrip("\n").split("\t")
        
        for i in range(len(raw_data)):
            data[header[i]] = raw_data[i]
        node = Node(data)
        coordinate = data["UniProtKB ID"]
        connection_to_other_graph = {"CSA_functional_sites": [data["PDB ID"]]}
        graph_manager.add_node_to_graph("CSA_protein_index", coordinate, node, set(), connection_to_other_graph)

In [4]:
x = graph_manager.get_graph("CSA_protein_index")

print(ray.get(ray.get(x.get_inter_graph_connections.remote())['P00963']['CSA_functional_sites']))

y = graph_manager.get_graph("CSA_functional_sites")

print(ray.get(ray.get(y.get_inter_graph_connections.remote())['12as']['CSA_protein_index']))

def insert_msa_into_graph_clustal_omega(file):
    with open(file, "r") as infile:
        infile.readline() # get rid of that first line
        
        msa_position_counter = {}
        residue_counter = {}
        for line in infile:
            if line[0] == "\n" or line[0] == " ":
                continue

            data = line.rstrip("\n").split("     ")
            uniprot_id = data[0].split(".")[0]
            alignment = data[1]
            
            if uniprot_id not in msa_position_counter:
                msa_position_counter[uniprot_id] = 0
            
            for amino_acid in alignment:
                neighbors = []
                if msa_position_counter[uniprot_id] != 0:
                    neighbors.append(msa_position_counter[uniprot_id] - 1)
                neighbors.append(msa_position_counter[uniprot_id] + 1)
                
                node = Node(amino_acid)
                
                connection_to_other_graph = {}
                connection_to_other_graph["CSA_protein_index"] = [uniprot_id]
                connection_to_other_graph["MSA_position"] = [msa_position_counter[uniprot_id]]

                if amino_acid != "-":
                    if not uniprot_id in residue_counter:
                        residue_counter[uniprot_id] = 0
                        
                    connection_to_other_graph["amino_acid"] = [amino_acid]
                    connection_to_other_graph[uniprot_id + "_genome_graph"] = [residue_counter[uniprot_id]]
                    residue_counter[uniprot_id] += 1
                
                graph_manager.add_node_to_graph("clustal_omega_msa", 
                                                str(uniprot_id) + "\t" + str(msa_position_counter[uniprot_id]),
                                                node,
                                                neighbors,
                                                connection_to_other_graph)
                
                msa_position_counter[uniprot_id] += 1
                
insert_msa_into_graph_clustal_omega("/Users/DevinPetersohn/Desktop/ProjectData/clustal_omega_1agy.txt")

{'12as'}
{'P00963'}


/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:48: Warning: clustal_omega_msa is not yet in this Graph Collection. Creating...
  warnings.warn(str(graph_id) + " is not yet in this Graph Collection. Creating...", Warning)
/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:56: Warning: MSA_position is not yet in this Graph Collection. Creating...
  warnings.warn(str(new_conn) + " is not yet in this Graph Collection. Creating...", Warning)
/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:56: Warning: amino_acid is not yet in this Graph Collection. Creating...
  warnings.warn(str(new_conn) + " is not yet in this Graph Collection. Creating...", Warning)
/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:56: Warning: P63882_genome_graph is not yet in this Graph Collection. Creating...
  warnings.warn(str(new_conn) + " is not yet in this Graph Collection. Creating...", Warning)
/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:56: Warning: P63880_geno

In [5]:
x = graph_manager.get_graph("clustal_omega_msa")
print(ray.get(ray.get(x.get_inter_graph_connections.remote())['P63882\t100']["MSA_position"]))

{100}


In [6]:
def insert_msa_into_graph_fasta_aln(file, msa_program):
    with open(file, "r") as infile:
        
        for line in infile:
            if line[0] == '>':
                if len(line.split("|")) > 1:
                    uniprot_id = line.split("|")[3].split(".")[0]
                else:
                    uniprot_id = line.split(" ")[0].split(">")[1].split(".")[0]
                msa_position_counter = 0
                residue_counter = 0
            else:
                for amino_acid in line:
                    neighbors = []
                    if msa_position_counter != 0:
                        neighbors.append(msa_position_counter - 1)
                    neighbors.append(msa_position_counter + 1)

                    node = Node(amino_acid)

                    connection_to_other_graph = {}
                    connection_to_other_graph["CSA_protein_index"] = [uniprot_id]
                    connection_to_other_graph["MSA_position"] = [msa_position_counter]

                    if amino_acid != "-":
                        connection_to_other_graph["amino_acid"] = [amino_acid]
                        connection_to_other_graph[uniprot_id + "_genome_graph"] = [residue_counter]
                        residue_counter += 1

                    graph_manager.add_node_to_graph(msa_program, 
                                                    str(uniprot_id) + "\t" + str(msa_position_counter),
                                                    node,
                                                    neighbors,
                                                    connection_to_other_graph)

                    msa_position_counter += 1
                    

In [7]:
insert_msa_into_graph_fasta_aln("/Users/DevinPetersohn/Desktop/ProjectData/muscle_1agy.txt", "muscle_msa")

/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:48: Warning: muscle_msa is not yet in this Graph Collection. Creating...
  warnings.warn(str(graph_id) + " is not yet in this Graph Collection. Creating...", Warning)


In [8]:
x = graph_manager.get_graph("muscle_msa")
print(ray.get(ray.get(x.get_inter_graph_connections.remote())['P63882\t100']["P63882_genome_graph"]))

{69}


In [9]:
insert_msa_into_graph_fasta_aln("/Users/DevinPetersohn/Desktop/ProjectData/cobalt_1agy.txt", "cobalt_msa")

/Users/DevinPetersohn/software_builds/g_graph/graphlib.py:48: Warning: cobalt_msa is not yet in this Graph Collection. Creating...
  warnings.warn(str(graph_id) + " is not yet in this Graph Collection. Creating...", Warning)


In [10]:
x = graph_manager.get_graph("cobalt_msa")
print(ray.get(ray.get(x.get_inter_graph_connections.remote())['P63882\t100']["P63882_genome_graph"]))

Remote function get_inter_graph_connections failed with:

Traceback (most recent call last):
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 786, in _process_task
    self._store_outputs_in_objstore(return_object_ids, outputs)
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 713, in _store_outputs_in_objstore
    self.put_object(objectids[i], outputs[i])
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 355, in put_object
    self.store_and_register(object_id, value)
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 290, in store_and_register
    object_id.id()), self.serialization_context)
  File "plasma.pyx", line 395, in pyarrow.plasma.PlasmaClient.put
  File "plasma.pyx", line 301, in pyarrow.plasma.PlasmaClient.create
  File "error.pxi", line 93, in pyarrow.lib.check_status

RayGetError: Could not get objectid ObjectID(669c66fe4da82c77ca3da0943d520c99d0076ab3). It was created by remote function [31mget_inter_graph_connections[39m which failed with:

Remote function [31mget_inter_graph_connections[39m failed with:

Traceback (most recent call last):
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 786, in _process_task
    self._store_outputs_in_objstore(return_object_ids, outputs)
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 713, in _store_outputs_in_objstore
    self.put_object(objectids[i], outputs[i])
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 355, in put_object
    self.store_and_register(object_id, value)
  File "/Users/DevinPetersohn/anaconda/envs/python36/lib/python3.6/site-packages/ray/worker.py", line 290, in store_and_register
    object_id.id()), self.serialization_context)
 

In [ ]:
print(1)